# Анализ развлекательного приложения Procrastinate Pro+.


В нашем распоряжении данные о пользователях приложения , привлечённых с 1 мая по 27 октября 2019 года:
- лог сервера с данными об их посещениях,
- выгрузка их покупок за этот период,
- рекламные расходы.

Цель проекта - проанализировать рекламные компании, понять почему они не успешны и помочь компании выйти в плюс. 

Наша работа будет состоять из 6 шагов:

1. Загрузка данных
2. Предобработка данных
3. Исследовательский анализ данных
4. Оценка маркетинговых показателей
5. Анализ окупаемости рекламы
6. Общий вывод

В итоге работы мы получим информацию о том, откуда приходят пользователи и какими устройствами они пользуются, сколько стоит привлечение пользователей из различных рекламных каналов, сколько денег приносит каждый клиент, когда расходы на привлечение клиента окупаются,
какие факторы мешают привлечению клиентов. Также мы сформулируем рекомендации для отдела маркетинга для дальнейшей работы и выхода компании в плюс.

### Загрузим данные и подготовим их к анализу

In [1]:
#Импортируем все необходимые библиотеки.
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
#Откроем файл visits.
try:
    visits = pd.read_csv('visits_info_short.csv')
except:
    visits = pd.read_csv('/datasets/visits_info_short.csv')
    
visits.head(5)

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/visits_info_short.csv'

In [ ]:
visits.info()

In [ ]:
#Проверим на наличие пропусков
visits.isna().sum()

In [ ]:
#Проверим на наличие дубликатов
visits.duplicated().sum()

В файле visits хранится информация о посещениях сайта. Под каким ID зашел пользователь, с какого региона и девайса. По какому каналу пришел и когда началась и закончилась сессия клиента. В файле 309901 строка, пропусков и явных дубликатов нет. Нам средует привести данные в столбце Session Start и Session End к типу даты pandas. Также приведем названия столбцов к нижнему регистру и заменим пробелы на нижнее подчеркивание. 

In [ ]:
#Изменим название столбцов 
visits.columns = visits.columns.str.lower()
visits.columns = ['user_id', 'region', 'device', 'channel', 'session_start', 'session_end']
visits.head(5)

In [ ]:
#Приведем столбец session_start и session_end к типу даты pandas
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
visits.info()

In [ ]:
#Откроем файл orders.
try:
    orders = pd.read_csv('orders_info_short.csv')
except:
    orders = pd.read_csv('/datasets/orders_info_short.csv')
    
orders.head(5)

In [ ]:
orders.info()

In [ ]:
#Проверим на наличие пропусков
orders.isna().sum()

In [ ]:
#Проверим на наличие дубликатов
orders.duplicated().sum()

В файле visits хранится информация о заказах. Под каким ID пользователь что-то купил, когда была совершена покупка и в какую цену был товар или услуга. В файле 40212 строки, пропусков и явных дубликатов нет. Нам средует привести данные в столбце event_dt к типу даты pandas. Также приведем названия столбцов к нижнему регистру и заменим пробелы на нижнее подчеркивание. 

In [ ]:
#Изменим название столбцов 
orders.columns = orders.columns.str.lower()
orders.columns = ['user_id', 'event_dt', 'revenue']
orders.head(5)

In [ ]:
#Приведем столбец event_dt к типу даты pandas
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
orders.info()

In [ ]:
#Откроем файл costs.
try:
    costs = pd.read_csv('costs_info_short.csv')
except:
    costs = pd.read_csv('/datasets/costs_info_short.csv')
    
costs.head(5)

In [ ]:
costs.info()

In [ ]:
#Проверим на наличие пропусков
costs.isna().sum()

In [ ]:
#Проверим на наличие дубликатов
costs.duplicated().sum()

В файле costs хранится информация о о расходах на рекламу. Дата проведения рекламной кампании, идентификатор рекламного источника, расходы на эту кампанию. В файле 1800 строк, пропусков и явных дубликатов нет. Нам средует привести данные в столбце dt к типу даты pandas. Также приведем названия столбцов к нижнему регистру.

In [ ]:
#Изменим название столбцов 
costs.columns = costs.columns.str.lower()
costs.head(5)

In [ ]:
#Приведем столбец event_dt к типу даты pandas
costs['dt'] = pd.to_datetime(costs['dt'])
costs.info()

In [ ]:
# добавим столбец month и week
costs['week'] = costs['dt'].dt.isocalendar().week
costs['month'] = costs['dt'].dt.month
costs

In [ ]:
costs['dt'] = pd.to_datetime(costs['dt']).dt.date

В ходе предобработки мы привели к нижнему регистру названия столбцов. Также изменили тип данных в столбце event_dt, dt, session_start, session_end на тип даты pandas. Явных дубликатов или пропусков в данных не обнаружено. 

## Зададим функции для расчёта и анализа LTV, ROI, удержания и конверсии.

Это функции для вычисления значений метрик:

- get_profiles() — для создания профилей пользователей,
- get_retention() — для подсчёта Retention Rate,
- get_conversion() — для подсчёта конверсии,
- get_ltv() — для подсчёта LTV.

А также функции для построения графиков:

- filter_data() — для сглаживания данных,
- plot_retention() — для построения графика Retention Rate,
- plot_conversion() — для построения графика конверсии,
- plot_ltv_roi — для визуализации LTV и ROI.

In [ ]:
# функция для создания пользовательских профилей

def get_profiles(sessions, orders, costs):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    costs = costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    costs['acquisition_cost'] = costs['costs'] / costs['unique_users']
    
    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles


In [ ]:
# функция для расчёта удержания или Retention Rate

def get_retention(
    profiles,
    sessions,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # добавляем столбец payer в передаваемый dimensions список
    dimensions = ['payer'] + dimensions

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        sessions[['user_id', 'session_start']], on='user_id', how='left'
    )
    result_raw['lifetime'] = (
        result_raw['session_start'] - result_raw['first_ts']
    ).dt.days

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу удержания
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # получаем таблицу динамики удержания
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [ ]:
# функция для визуализации удержания

def plot_retention(retention, retention_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 10))

    # исключаем размеры когорт и удержание первого дня
    retention = retention.drop(columns=['cohort_size', 0])
    # в таблице динамики оставляем только нужный лайфтайм
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # если в индексах таблицы удержания только payer,
    # добавляем второй признак — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # в таблице графиков — два столбца и две строки, четыре ячейки
    # в первой строим кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание платящих пользователей')

    # во второй ячейке строим кривые удержания неплатящих
    # вертикальная ось — от графика из первой ячейки
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Удержание неплатящих пользователей')

    # в третьей ячейке — динамика удержания платящих
    ax3 = plt.subplot(2, 2, 3)
    # получаем названия столбцов для сводной таблицы
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания платящих пользователей на {}-й день'.format(
            horizon
        )
    )

    # в чётвертой ячейке — динамика удержания неплатящих
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # фильтруем данные и строим график
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Дата привлечения')
    plt.title(
        'Динамика удержания неплатящих пользователей на {}-й день'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show()

In [ ]:
# функция для расчёта конверсии

def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # рассчитываем лайфтайм для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users' 
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # для таблицы динамики конверсии убираем 'cohort' из dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [ ]:
# функция для визуализации конверси

def plot_conversion(conversion, conversion_history, horizon, window=7):

    # задаём размер сетки для графиков
    plt.figure(figsize=(15, 5))

    # исключаем размеры когорт
    conversion = conversion.drop(columns=['cohort_size'])
    # в таблице динамики оставляем только нужный лайфтайм
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # первый график — кривые конверсии
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('Конверсия пользователей')

    # второй график — динамика конверсии
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [		
        # столбцами сводной таблицы станут все столбцы индекса, кроме даты
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика конверсии пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

In [ ]:
# функция для расчёта LTV и ROI

def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
  
    # добавляем данные о покупках в профили
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    
    # рассчитываем лайфтайм пользователя для каждой покупки
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    
    # группируем по cohort, если в dimensions ничего нет
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция группировки по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        # строим «треугольную» таблицу выручки
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # находим сумму выручки с накоплением
        result = result.fillna(0).cumsum(axis=1)
        # вычисляем размеры когорт
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # объединяем размеры когорт и таблицу выручки
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # считаем LTV: делим каждую «ячейку» в строке на размер когорты
        result = result.div(result['cohort_size'], axis=0)
        # исключаем все лайфтаймы, превышающие горизонт анализа
        result = result[['cohort_size'] + list(range(horizon_days))]
        # восстанавливаем размеры когорт
        result['cohort_size'] = cohort_sizes

        # собираем датафрейм с данными пользователей и значениями CAC, 
        # добавляя параметры из dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # считаем средний CAC по параметрам из dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # считаем ROI: делим LTV на CAC
        roi = result.div(cac['cac'], axis=0)

        # удаляем строки с бесконечным ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # восстанавливаем размеры когорт в таблице ROI
        roi['cohort_size'] = cohort_sizes

        # добавляем CAC в таблицу ROI
        roi['cac'] = cac['cac']

        # в финальной таблице оставляем размеры когорт, CAC
        # и ROI в лайфтаймы, не превышающие горизонт анализа
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # возвращаем таблицы LTV и ROI
        return result, roi

    # получаем таблицы LTV и ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # для таблиц динамики убираем 'cohort' из dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицы динамики LTV и ROI
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # сырые данные
        result_grouped,  # таблица LTV
        result_in_time,  # таблица динамики LTV
        roi_grouped,  # таблица ROI
        roi_in_time,  # таблица динамики ROI
    )

In [ ]:
# функция для сглаживания фрейма

def filter_data(df, window):
    # для каждого столбца применяем скользящее среднее
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df

# функция для визуализации LTV и ROI

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=7):

    # задаём сетку отрисовки графиков
    plt.figure(figsize=(20, 20))

    # из таблицы ltv исключаем размеры когорт
    ltv = ltv.drop(columns=['cohort_size'])
    # в таблице динамики ltv оставляем только нужный лайфтайм
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # стоимость привлечения запишем в отдельный фрейм
    cac_history = roi_history[['cac']]

    # из таблицы roi исключаем размеры когорт и cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # в таблице динамики roi оставляем только нужный лайфтайм
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # первый график — кривые ltv
    ax1 = plt.subplot(3, 2, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('LTV')

    # второй график — динамика ltv
    ax2 = plt.subplot(3, 2, 2, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # третий график — динамика cac
    ax3 = plt.subplot(3, 2, 3, sharey=ax1)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Дата привлечения')
    plt.title('Динамика стоимости привлечения пользователей')
    
    # четвёртый график — кривые roi
    ax4 = plt.subplot(3, 2, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.legend()
    plt.xlabel('Лайфтайм')
    plt.title('ROI')

    # пятый график — динамика roi
    ax5 = plt.subplot(3, 2, 5, sharey=ax4)
    # столбцами сводной таблицы станут все столбцы индекса, кроме даты
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Дата привлечения')
    plt.title('Динамика ROI пользователей на {}-й день'.format(horizon))

    plt.tight_layout()
    plt.show()

### Исследовательский анализ данных

- Составьте профили пользователей. Определите минимальную и максимальную даты привлечения пользователей.
- Выясните, из каких стран пользователи приходят в приложение и на какую страну приходится больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих из каждой страны.
- Узнайте, какими устройствами пользуются клиенты и какие устройства предпочитают платящие пользователи. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого устройства.
- Изучите рекламные источники привлечения и определите каналы, из которых пришло больше всего платящих пользователей. Постройте таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения.

После каждого пункта сформулируйте выводы.

In [ ]:
# составим профили пользователей
profiles = get_profiles(visits, orders, costs)
profiles.head(10)

In [ ]:
# определим минимальную и максимальную даты привлечения пользователей

observation_date = datetime(2019, 11, 1).date()  # момент анализа
analysis_horizon = 14  # горизонт анализа

# посчитаем минимальную дату привлечения
min_analysis_date = profiles['dt'].min()

# посчитаем максимальную дату привлечения
max_analysis_date = profiles['dt'].max()

print(f'Минимальная дата привлечения пользователей: {min_analysis_date}')
print(f'Максимальная дата привлечения пользователей: {max_analysis_date}')

Выясним, из каких стран пользователи приходят в приложение и на какую страну приходится больше всего платящих пользователей.

In [ ]:
country =  (profiles
           .pivot_table(
                        index='region',
                        columns='payer',
                        values='user_id',
                        aggfunc='count')
           .rename(columns={True: 'payer', False: 'not_payer'})
           .sort_values(by='payer', ascending=False)
          )

In [ ]:
country['payer_share'] = (country.payer / (country.not_payer + country.payer) * 100).round(2)
country

Построим  таблицу, отражающую количество пользователей и долю платящих из каждой страны.

In [ ]:
fig, ax = plt.subplots(figsize=(18, 3))
country[['payer', 'not_payer']].plot(kind='barh', stacked=True, ax=ax, alpha=0.4)

ax.legend(bbox_to_anchor=(1.0, 1.0))
ax.set_xlabel('Количество пользователей')
ax.set_ylabel('Страна')
ax.set_title('Привлеченные пользователи по странам',loc='left')
plt.show()

Судя по графику наибольшее количество пользователей приложения из США. Далее идет Великобритания, Франция и Германия. Наибольший процент платёжеспособных пользователей также в США. Доля людей, которые платят за приложение в США составляет 6,9%. Несмотря на то, что пользователей в Германии меньше всего, процент платящих клиентов там 4,11%, что занимает вторую строчку среди всех стран. 

Узнаем, какими устройствами пользуются клиенты и какие устройства предпочитают платящие пользователи.

In [ ]:
device =  (profiles
           .pivot_table(
                        index='device',
                        columns='payer',
                        values='user_id',
                        aggfunc='count')
           .rename(columns={True: 'payer', False: 'not_payer'})
           .sort_values(by='payer', ascending=False)
          )

In [ ]:
device['payer_share'] = (device.payer / (device.not_payer + device.payer) * 100).round(2)
device

Построим таблицу, отражающую количество пользователей и долю платящих для каждого устройства.

In [ ]:
fig, ax = plt.subplots(figsize=(18, 3))
device[['payer', 'not_payer']].plot(kind='barh', stacked=True, ax=ax, alpha=0.4)

ax.legend(bbox_to_anchor=(1.0, 1.0))
ax.set_xlabel('Количество пользователей')
ax.set_ylabel('Устройство')
ax.set_title('Привлеченные пользователи в разрезе устройств',loc='left')
plt.show()

Большенство клиентов предпочитают использовать iPhone, далее по популярности идут утройства на Android, Mac и PC. Доля платящих клиентов больше у Mac и iPhone, она составляет 6,36% и 6,21% соответственно. 

Изучим рекламные источники привлечения и определим каналы, из которых пришло больше всего платящих пользователей.

In [ ]:
channel =  (profiles
           .pivot_table(
                        index='channel',
                        columns='payer',
                        values='user_id',
                        aggfunc='count')
           .rename(columns={True: 'payer', False: 'not_payer'})
           .sort_values(by='payer', ascending=False)
          )

In [ ]:
channel['payer_share'] = (channel.payer / (channel.not_payer + channel.payer) * 100).round(2)
channel

Построим таблицу, отражающую количество пользователей и долю платящих для каждого канала привлечения.

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
channel[['payer', 'not_payer']].plot(kind='barh', stacked=True, ax=ax, alpha=0.4)

ax.legend(bbox_to_anchor=(1.0, 1.0))
ax.set_xlabel('Количество пользователей')
ax.set_ylabel('Каналы привлечения')
ax.set_title('Привлеченные пользователи в разрезе каналов',loc='left')
plt.show()

Большенство клиентов приходит в приложение органически, но доля платящих клиентов там составляет лишь 2%. Самые успешные каналы привлечения пользователей это FaceBoom и TipTop. Доля платящих клиентов, приходящих из этих каналов составляет 12,2% и 9,6% соответственно. 

<b>Вывод</b>

Проведя исследовательский анализ данных мы можем сделать вывод, что приложение популярно в США, клиенты предпочитают использовать iPhone. Наиболее успешными каналами привлечения пользователей можно назвать FaceBoom и TipTop.

### Маркетинг

- Посчитайте общую сумму расходов на маркетинг.
- Выясните, как траты распределены по рекламным источникам, то есть сколько денег потратили на каждый источник.
- Постройте визуализацию динамики изменения расходов во времени (по неделям и месяцам) по каждому источнику. Постарайтесь отразить это на одном графике.
- Узнайте, сколько в среднем стоило привлечение одного пользователя (CAC) из каждого источника. Используйте профили пользователей.

Напишите промежуточные выводы.

In [ ]:
total_ad_spend = costs.costs.sum().round()
print('Общая сумма расходов на маркетинг:', total_ad_spend)

In [ ]:
# посмотрим, как распределились траты по рекламным источникам
costs.pivot_table(index='channel',values='costs',aggfunc='sum').sort_values(by='costs', ascending=False)

Больше всего тратили на платформы TipTop и FaceBoom. На TipTop тратили на 70% больше чем на FaceBoom, но FaceBoom принёс больше платёжеспособных клиентов приложению.  

In [ ]:
# построим график еженедельных затрат по каналам

fig, ax = plt.subplots(figsize=(16, 6))

(costs
 .pivot_table(
    index=costs.week,
    values='costs',
    aggfunc='sum',
    columns='channel'
)
 .plot(ax=ax, stacked=True)
)

ax.set_title('Затраты на привлечение пользователей в разрезе каналов', loc='left')
ax.set_ylabel('Затраты на привлечение в неделю')
ax.set_xlabel('неделя')
plt.show()

Судя по графику мы видим стабильный рост затрат на привлечение пользователей. Также почти по всем каналам привлечения мы можем заметить всплеск затрат на 23, 26 и 39 неделе. Возможно это связанно с какими-то существенными обновлениями приложения.  

In [ ]:
# построим график ежемесячных затрат по каналам

fig, ax = plt.subplots(figsize=(16, 6))

(costs
 .pivot_table(
    index=costs.month,
    values='costs',
    aggfunc='sum',
    columns='channel'
)
 .plot(ax=ax, stacked=True)
)

ax.set_title('Затраты на привлечение пользователей в разрезе каналов', loc='left')
ax.set_ylabel('Затраты на привлечение в месяц')
ax.set_xlabel('месяц')
plt.show()

Смотря на ежемесячный график затрат на привлечение клиентов мы также наблюдаем стабильный рост, но без резких скачков. 

Рассчитаем сколько в среднем стоило привлечение одного пользователя из каждого источника(CAC). 
Мы исключили из этого исследования канал Organic, так как на него компания не тратит деньги.  

In [ ]:
cac_person_mean = (profiles
              .query('channel != "organic"')[['user_id', 'acquisition_cost']]
              .drop_duplicates()
              .agg({'acquisition_cost': 'mean'})
             )
print('Средняя цена привлечения одного пользователя:', cac_person_mean['acquisition_cost'])

In [ ]:
cac_channel = (profiles
       .pivot_table(index = 'channel',
                    values = 'acquisition_cost',
                    aggfunc='mean')
       .sort_values(by='acquisition_cost', ascending=False)
       .rename(columns={'acquisition_cost': 'cac'})
      )
cac_channel

Дороже всего обходятся пользователи привлченные из TipTop и FaceBoom.

### Оцените окупаемость рекламы

Используя графики LTV, ROI и CAC, проанализируйте окупаемость рекламы. Считайте, что на календаре 1 ноября 2019 года, а в бизнес-плане заложено, что пользователи должны окупаться не позднее чем через две недели после привлечения. Необходимость включения в анализ органических пользователей определите самостоятельно.

- Проанализируйте окупаемость рекламы c помощью графиков LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проверьте конверсию пользователей и динамику её изменения. То же самое сделайте с удержанием пользователей. Постройте и изучите графики конверсии и удержания.
- Проанализируйте окупаемость рекламы с разбивкой по устройствам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по странам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Проанализируйте окупаемость рекламы с разбивкой по рекламным каналам. Постройте графики LTV и ROI, а также графики динамики LTV, CAC и ROI.
- Ответьте на такие вопросы:
    - Окупается ли реклама, направленная на привлечение пользователей в целом?
    - Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
    - Чем могут быть вызваны проблемы окупаемости?

Напишите вывод, опишите возможные причины обнаруженных проблем и промежуточные рекомендации для рекламного отдела.

In [ ]:
# так пользователи пришедшие органически в приложение ничего не стоили компании, удалим их из анализа для более честного анализа
profiles = profiles.query('channel != "organic"')

In [ ]:
# Проанализируем окупаемость рекламы
# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, analysis_horizon)

# строим графики
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, analysis_horizon)

Судя по графикам, можно сделать вывод:
- На 14 день LTV равняется 0,9 центам. В среднем данный показатель не сильно меняется на протяжении всего анализируемого времени.  
- Стоимость привлечения пользователей растёт с каждым днем, поэтому и застраты на маркетинг вырастают с каждым днём. 
- Окупаемость рекламных компаний сначала растёт, но так и не переступает порог окупаемости. 
- С мая по середину июня мы видим, что реклама окупалась за 2 недели, но потом был скачёк вниз и реклама перестала окупаться за этот период. 

In [ ]:
# Проанализируем окупаемость рекламы с разбивкой по странам
# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, analysis_horizon, dimensions = ['region'])
# строим графики
plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, analysis_horizon, window=14)

- Судя по графику, самый высокий LTV — у пользователей из США. Затем идут Великобритания и Германия. Пользователи из Франции приносят приложению меньше всего выручки.
- Сначала сумма привлечения пользователя в приложение во всём мире была примерно на одном уровне, но с июня пользователи из США стали сильно дороже обходиться платформе, чем пользователи из Европы. 
- Пользователи пришедшие с рынка США не окупаются вовсе и быстро уходят с платформы. 

In [ ]:
# Проанализируем окупаемость рекламы с разбивкой по устройствам
# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, analysis_horizon, dimensions = ['device'])
# строим графики
plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, analysis_horizon, window=14)

- LTV примерно находится на одном уровне у всех пользователей с разными девайсами. 
- Стоимость привлечения пользователей растёт с каждым днем для каждого пользователя, но больше у пользователей с девайсами от Apple. 
- Пользователи с iPhone и MAC лучше окупаются, чем остальные.  
- Сложнее всего окупаются пользователи которые исаользуют PC. 

In [ ]:
# Проанализируем окупаемость рекламы с разбивкой по каналам продаж
# считаем LTV и ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, analysis_horizon, dimensions = ['channel'])
# строим графики
plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, analysis_horizon, window=14)

In [ ]:
# На графиках сложно что то разобрать, поэтому построим тепловую карту LTV для наглядности. 
plt.figure(figsize=(35, 10))

sns.heatmap(
    ltv_grouped.drop(columns = ['cohort_size']), 
    annot=True, 
    fmt='.2%', 
    ax=plt.subplot(1, 2, 2)
            )
plt.title('Тепловая карта LTV с разбивкой по каналам привлечения')
plt.xlabel('Лайфтайм')
plt.ylabel('Каналы привлечения')
plt.show()

- Компании в которые вливают больше всего средств не окупаются (TipTop, FaceBoom, AdNonSense и lambdaMediaAds)
- Затраты на канал TipTop с каждым месяцем увеличиваются вдвое. 

In [ ]:
# Проверим конверсию пользователей и динамику её изменения
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, analysis_horizon)

plot_conversion(conversion_grouped, conversion_history, analysis_horizon)

Конверсия пользователей стабильно растёт на протяжении всего периода анализа. 

In [ ]:
# Проверим удержание пользователей и динамику её изменения
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, analysis_horizon)

plot_retention(retention_grouped, retention_history, analysis_horizon)

- Кривая удержания платящих пользователей выше, чем неплатящих,
- Обе кривые постепенно снижаются.

Рассмотрим также конверсию пользователей и динамику её изменения в разрезе стран и каналов продаж. 

In [ ]:
# Проверим конверсию пользователей и динамику её изменения в разрезе стран
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, analysis_horizon, dimensions = ['region'])

plot_conversion(conversion_grouped, conversion_history, analysis_horizon)

Судя по графику, конверсия пользователей из США больше в два раза, чем у Европейских пользователей.

In [ ]:
# Проверим удержание пользователей и динамику её изменения в разрезе стран
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, analysis_horizon, dimensions = ['region'])

plot_retention(retention_grouped, retention_history, analysis_horizon)

Однако если посмотреть на график удержания пользоватлей, США на последнем месте. Это значит, что пользователи из США быстро прихлдят в приложение, но так же стремительно с него и уходят. 

In [ ]:
# Проверим конверсию пользователей и динамику её изменения в разбивке по каналам привлечения
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, analysis_horizon, dimensions = ['channel'])

plot_conversion(conversion_grouped, conversion_history, analysis_horizon)

В среднем конверсия пользователей привлеченных с разных каналов находится в стагнации. Самая сильная конверсия у пользователей из FaceBoom, AdNonSense и lambdaMediaAds. 

In [ ]:
# Проверим удержание пользователей и динамику её изменения в разбивке по каналам привлечения
retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, analysis_horizon, dimensions = ['channel'])

plot_retention(retention_grouped, retention_history, analysis_horizon)

Если смотреть на график удержания пользоватлей, FaceBoom и AdNonSense на последнем месте. Это значит, что пользователи из этих каналов быстро прихлдят в приложение, но так же стремительно с него и уходят. Остальные каналы привлечение показывают более лучшие результаты удерживания платящих пользователей, в том сичисле и lambdaMediaAds, которая даёт хорошую конверсию. 
    
Что касается TipTop, данный канал показывает достаточно высокую конверсию и удержание пользователей, но к сожалению он слишком дорогой и совсем не окупается. 

- Окупается ли реклама, направленная на привлечение пользователей в целом?
    - Нет, реклама не окупается. 
- Какие устройства, страны и рекламные каналы могут оказывать негативное влияние на окупаемость рекламы?
    - Наиболее негативно на рекламную компанию влияют пользователи из США, которые обходятся компании сильно дороже европейскийх и быстрее уходят с приложения. Рекламные каналы TipTop и FaceBoom, на которые тратится львиная доля бюджета совсем не окупаются. Устройства в значительной мере не имеюбт значения на успешность рекламной компании, однако стоит одтавать предпочтение устройствам от компании Apple (iPhone, Mac). 
- Чем могут быть вызваны проблемы окупаемости?
    - Возможно главная проблема кроется в неправильно выбранном рынке продаж. Раз пользователи из США так дорого обходяться компании и совсем мало задерживаются, может лучше сделать упор на Европейский рынок?
        Следует далее более детально проанализировать рекламные компании исключив США, для подтверждения данной теории. 

## Вывод

В ходе работы были проанализированы данные о пользователей приложения Procrastinate Pro+, привлечённых с 1 мая по 27 октября 2019 года (лог сервера с данными об их посещениях, выгрузка их покупок за этот период, рекламные расходы). 
 
Компания несёт убытки, несмотря на огромные вложения в рекламу за последние 7 месяцев.

Мы рассмотрели факторы, которые могли на неэффективное привлечение клиентов в приложение и пришли к определенным выводам:
- Рекламные каналы TipTop и FaceBoom, на которые тратится львиная доля бюджета совсем не окупаются.
- Пользователи из США, на которых было потрачено больше всего буджета, не окупаются за 2 недели и быстро уходят с платформы. 
- Расходы на рекламный канал TipTop вырос каждый месяц увеличивался на 70%, тем самым ещё больше загонял компанию в убытки. 
- Стоимость привлечения пользователей растёт с каждым днем, поэтому и застраты на маркетинг вырастают с каждым днём.

В качестве рекомендаций для отдела маркетинга можно выделить следующее:
- Рассмотреть другие площадки для привлечение клиентов, например WahooNetBanner или RocketSuperAds. Они окупаются за 14ти дневный период, приносят пользователей и не требуют сильных вложений. 
- Обратить более тщательное внимание на Европейский рынок. Возможно в США сушествует более популярное и конкурентноспособное приложение, которое отвечает всем требованиям пользователй. А в Европе наоборот популярно приложение Procrastinate Pro+ и его там можно развить.   
- Сделать упор на развитие площадки для пользователй с iPhone и Mac. 
- Перестать вливать деньги в TipTop и FaceBoom
- Проанализировать другие странах. Возможно следует развивать это приложение не только в 4 странах, но и открыть новые направления. 

Если рассматривать всё таки рынок США, возможно стоит использовать другие каналы привлечения клиентов, а не только TipTop и FaceBoom.
Показатели канала FaceBoom не такие плохие, конверсия пользователей выше чем у других каналов, однако стоит поработать на удержание платящей аудитори. 